In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Dense, Flatten, Conv2D, MaxPooling2D, BatchNormalization, Dropout, Reshape, Concatenate, LeakyReLU
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model

In [ ]:
# Height and width refer to the size of the image
# Channels refers to the amount of color channels (red, green, blue)

image_dimensions = {'height':256, 'width':256, 'channels':3}

In [ ]:
# Create a Classifier class

class Classifier:
    def __init__():
        self.model = 0
    
    def predict(self, x):
        return self.model.predict(x)
    
    def fit(self, x, y):
        return self.model.train_on_batch(x, y)
    
    def get_accuracy(self, x, y):
        return self.model.test_on_batch(x, y)
    
    def load(self, path):
        self.model.load_weights(path)

In [ ]:
# Create a MesoNet class using the Classifier

class Meso4(Classifier):
    def __init__(self, learning_rate = 0.001):
        self.model = self.init_model()
        optimizer = Adam(lr = learning_rate)
        self.model.compile(optimizer = optimizer,
                           loss = 'mean_squared_error',
                           metrics = ['accuracy'])
    
    def init_model(self): 
        x = Input(shape = (image_dimensions['height'],
                           image_dimensions['width'],
                           image_dimensions['channels']))
        
        x1 = Conv2D(8, (3, 3), padding='same', activation = 'relu')(x)
        x1 = BatchNormalization()(x1)
        x1 = MaxPooling2D(pool_size=(2, 2), padding='same')(x1)
        
        x2 = Conv2D(8, (5, 5), padding='same', activation = 'relu')(x1)
        x2 = BatchNormalization()(x2)
        x2 = MaxPooling2D(pool_size=(2, 2), padding='same')(x2)
        
        x3 = Conv2D(16, (5, 5), padding='same', activation = 'relu')(x2)
        x3 = BatchNormalization()(x3)
        x3 = MaxPooling2D(pool_size=(2, 2), padding='same')(x3)
        
        x4 = Conv2D(16, (5, 5), padding='same', activation = 'relu')(x3)
        x4 = BatchNormalization()(x4)
        x4 = MaxPooling2D(pool_size=(4, 4), padding='same')(x4)
        
        y = Flatten()(x4)
        y = Dropout(0.5)(y)
        y = Dense(16)(y)
        y = LeakyReLU(alpha=0.1)(y)
        y = Dropout(0.5)(y)
        y = Dense(1, activation = 'sigmoid')(y)

        return Model(inputs = x, outputs = y)

In [ ]:
# Instantiate a MesoNet model with pretrained weights
meso = Meso4()
meso.load('/content/weights/Meso4_DF')

In [ ]:
# Prepare image data

# Rescaling pixel values (between 1 and 255) to a range between 0 and 1
dataGenerator = ImageDataGenerator(rescale=1./255)

# Instantiating generator to feed images through the network
generator = dataGenerator.flow_from_directory(
    '/content/data/',
    target_size=(256, 256),
    batch_size=1,
    class_mode='binary')

In [ ]:
# Checking class assignment
generator.class_indices

In [ ]:
#There are 6 categories
realWithHighConfidence = []
realWithMediumConfidence = []
realWithLowConfidence = []
fakeWithLowConfidence = []
fakeWithMediumConfidence = []
fakeWithHighConfidence = []
count = 1
for i in range(0,len(generator.labels)):
  print(count)
  X, y = generator.next()
  prediction = meso.predict(X)[0][0]
  if prediction >= 0 and prediction <= 5/30:
    fakeWithHighConfidence.append(X)
  elif prediction > 5/30 and prediction <= 10/30:
    fakeWithMediumConfidence.append(X)
  elif prediction > 10/30 and prediction <= 15/30:
    fakeWithLowConfidence.append(X)
  elif prediction > 15/30 and prediction <= 20/30:
    realWithLowConfidence.append(X)
  elif prediction > 20/30 and prediction <= 25/30:
    realWithMediumConfidence.append(X)
  elif prediction > 25/30 and prediction <= 30/30:
    realWithHighConfidence.append(X)
  count += 1
  

In [ ]:
from tabulate import tabulate

fakeHigh = (len(fakeWithHighConfidence))
fakeMedium = (len(fakeWithMediumConfidence))
fakeLow = (len(fakeWithLowConfidence))
realLow = (len(realWithLowConfidence))
realMedium = (len(realWithMediumConfidence))
realHigh = (len(realWithHighConfidence))
totalImages = len(generator.labels)

#categories
fakeHighPercent = str(round(fakeHigh/totalImages*100,2)) + '%'
fakeMediumPercent = str(round(fakeMedium/totalImages*100,2))+'%'
fakeLowPercent = str(round(fakeLow/totalImages*100,2))+'%'
realLowPercent = str(round(realLow/totalImages*100,2))+'%'
realMediumPercent = str(round(realMedium/totalImages*100,2))+'%'
realHighPercent = str(round(realHigh/totalImages*100,2))+'%'

table1 = [['Category','Number of images in this category','total number of images','percent of total'],
          ['fake with high confidence',fakeHigh,len(generator.labels),fakeHighPercent],
          ['fake with medium confidence', fakeMedium, len(generator.labels), fakeMediumPercent],
          ['fake with low confidence', fakeLow, len(generator.labels), fakeLowPercent],
          ['real with low confidence', realLow, len(generator.labels), realLowPercent],
          ['real with medium confidence', realMedium, len(generator.labels), realMediumPercent],
          ['real with high confidence', realHigh, len(generator.labels), realHighPercent]]
print(tabulate(table1,headers='firstrow',tablefmt='fancy_grid'))


#Predicted labels
fakeTotal = fakeLow+fakeMedium+fakeHigh
realTotal = realLow+realMedium+realHigh

FakePercent = str(round(fakeTotal/totalImages*100,2))+'%'
RealPercent = str(round(realTotal/totalImages*100,2))+'%'

table2 = [['Predicted Label','Number of Images for this label','Total number of Images','Percent of total'],
         ['Real', realTotal, len(generator.labels), RealPercent],
         ['Fake', fakeTotal, len(generator.labels), FakePercent]]
print(tabulate(table2,headers='firstrow',tablefmt='fancy_grid'))

#Confidences
lowConfidence = fakeLow + realLow
mediumConfidence = fakeMedium + realMedium
highConfidence = fakeHigh + realHigh

lowConfidencePercent = str(round(lowConfidence/totalImages*100,2))+'%'
mediumConfidencePercent = str(round(mediumConfidence/totalImages*100,2))+'%'
highConfidencePercent = str(round(highConfidence/totalImages*100,2))+'%'

table3 = [['Confidence', 'Number of images for this confidence interval', 'total number of images', 'percent of total'],
          ['low confidence', lowConfidence, totalImages, lowConfidencePercent],
          ['medium confidence', mediumConfidence, totalImages, mediumConfidencePercent],
          ['high confidence', highConfidence, totalImages, highConfidencePercent]]
print(tabulate(table3,headers='firstrow',tablefmt='fancy_grid'))

